In [107]:
import csv
from elasticsearch import helpers, Elasticsearch
import os
from __future__ import print_function
import glob
import json
import sys
import requests
import time
import re
import datetime
import pandas as pd
import numpy as np

ImportError: Missing required dependencies ['numpy']

In [3]:
es = Elasticsearch(['localhost:9200'], port=9200)

In [4]:
es.info()

{u'cluster_name': u'elasticsearch',
 u'cluster_uuid': u'YSCvebMxQfy03EK_8Hbs8A',
 u'name': u'com-desktop',
 u'tagline': u'You Know, for Search',
 u'version': {u'build_date': u'2021-03-06T05:54:38.141101Z',
  u'build_flavor': u'default',
  u'build_hash': u'3e5a16cfec50876d20ea77b075070932c6464c7d',
  u'build_snapshot': False,
  u'build_type': u'deb',
  u'lucene_version': u'8.7.0',
  u'minimum_index_compatibility_version': u'6.0.0-beta1',
  u'minimum_wire_compatibility_version': u'6.8.0',
  u'number': u'7.11.2'}}

In [42]:
doc = """{
  "settings": {
    "analysis": {
    "tokenizer":
    {
    "nori_tokenizer":
        {
            "type":"nori_tokenizer"
        }
    }
    ,
      "analyzer": {
        "nori": {
          "tokenizer": "nori_tokenizer"
        }
      }
    }
  },
  "mappings": {
    "properties": {
        "businessId":{
        "type" : "long"
        },
      "body": {
        "type": "text",
        "analyzer": "nori"       
        },
        "rating":{
            "type":"long"
        },
        "name":{
            "type":"text",
            "analyzer" : "nori"
        },
        "nickName":{
            "type":"text",
            "analyzer":"nori"
        }
        
    }
  }
}"""
es.indices.create(index="google",body= json.loads(doc))

{u'acknowledged': True, u'index': u'google', u'shards_acknowledged': True}

In [43]:
doc = """{
  "settings": {
    "analysis": {
    "tokenizer":
    {
    "nori_tokenizer":
        {
            "type":"nori_tokenizer"
        }
    }
    ,
      "analyzer": {
        "nori": {
          "tokenizer": "nori_tokenizer"
        }
      }
    }
  },
  "mappings": {
    "properties": {
        "businessId":{
        "type" : "long"
        },
      "category": {
        "type": "text",
        "analyzer": "nori"       
        },
        "address":{
            "type":"text",
            "analyzer":"nori"
        },
        "roadAddress":{
            "type":"text",
            "analyzer":"nori"
        },
        "menu":{
        "type":"text",
        "analyzer":"nori"
        },
        "info":{
        "type":"text",
        "analyzer":"nori"
        }
        ,
        "name":{
            "type":"text",
            "analyzer" : "nori"
        },
        "nickName":{
            "type":"text",
            "analyzer":"nori"
        }
        
    }
  }
}"""
es.indices.create(index="naver",body= json.loads(doc))

{u'acknowledged': True, u'index': u'naver', u'shards_acknowledged': True}

In [44]:
doc = """{
  "settings": {
    "analysis": {
    "tokenizer":
    {
    "nori_tokenizer":
        {
            "type":"nori_tokenizer"
        }
    }
    ,
      "analyzer": {
        "nori": {
          "tokenizer": "nori_tokenizer"
        }
      }
    }
  },
  "mappings": {
    "properties": {
        "businessId":{
        "type" : "long"
        },
        "rating":{
        "type":"long"
        }
        ,
        "body":{
            "type":"text",
            "analyzer" : "nori"
        },
        "nickName":{
            "type":"text",
            "analyzer":"nori"
        }
        
    }
  }
}"""
es.indices.create(index="store",body= json.loads(doc))

{u'acknowledged': True, u'index': u'store', u'shards_acknowledged': True}

In [16]:
json.loads(doc)

{u'mappings': {u'properties': {u'body': {u'analyzer': u'nori',
    u'type': u'text'},
   u'businessId': {u'type': u'long'},
   u'name': {u'analyzer': u'nori', u'type': u'text'},
   u'nickName': {u'analyzer': u'nori', u'type': u'text'},
   u'rating': {u'type': u'long'}}},
 u'settings': {u'analysis': {u'analyzer': {u'nori': {u'tokenizer': u'nori_tokenizer'}},
   u'tokenizer': {u'nori_tokenizer': {u'type': u'nori_tokenizer'}}}}}

In [45]:
with open('네이버스토어.csv') as f:
    rdr = csv.reader(f)
    for idx,line in enumerate(rdr):
        if idx is not 0:
            doc1 = {"businessId":float(line[0]),'nickName':line[2], 'body': line[3],    'rating': float(line[1])}                  
            es.index(index="store", body=doc1)
    f.close()    

In [46]:
with open('구글리뷰.csv') as f:
    rdr = csv.reader(f)
    for idx,line in enumerate(rdr):
        if idx is not 0:
            doc1 = {"businessId": float(line[0]), 'name':line[1], 'nickName':line[2], 'body': line[4],    'rating': float(line[3])}                  
            es.index(index="google", body=doc1)
    f.close()    

In [47]:
with open('네이버리뷰.csv') as f:
    rdr = csv.reader(f)
    for idx,line in enumerate(rdr):
        if idx is not 0:
            doc1 = {"businessId": float(line[0]), 'name':line[1], 'category':line[2], 'address':line[3],'roadAddress':line[4],'menu':line[5],'info':line[6]}                  
            es.index(index="naver", body=doc1)
    f.close()    

In [98]:
#후기가 어디 가게인지
#위치별로 보기
#종목별로 보기
#가게별로 보기
#점수별로 보기

def search():
    body = {
        'size':5000,
        'query':
        {
            'match':
            {    
            }
        }
    }
    while 1:
        case = raw_input("볼 목록을 고르세요. \n 1: 후기 \n 2: 위치별 후기 \n 3: 종목별 후기 \n 4: 가게별 후기 \n 5: 점수별 후기 \n 나가려면 exit")
        
        if case is "1":
            query = raw_input("질의어를 입력하세요.")
                              
            body['query']['match']['body'] = query

            results = es.search(index="google", body=body)
            for result in results['hits']['hits']:
                print("명칭: ",result['_source']['name'])
                print("점수: ",result['_source']['rating'])
                print("후기글: ",result['_source']['body'].encode("utf-8").replace('\n',' '))
                print("=============================================================================================================")
            
            del body['query']['match']['body']
            
        elif case is "2":
            pos = raw_input("위치를 입력하세요\n")
            body['query']['match']['address'] = pos
            
            results = es.search(index="naver", body=body)
            businessId = {}
            for result in results['hits']['hits']:
                businessId[result['_source']['businessId']] = True
            del body['query']['match']['address']
            
            
            query = raw_input("질의어를 입력하세요.")
            body['query']['match']['body'] = query

            results = es.search(index="google", body=body)
            for result in results['hits']['hits']:
                if result['_source']['businessId'] in businessId:
                    print("명칭: ",result['_source']['name'])
                    print("점수: ",result['_source']['rating'])
                    print("후기글: ",result['_source']['body'].encode("utf-8").replace('\n',' '))
                    print("=============================================================================================================")
            del body['query']['match']['body']
            
            
        elif case is "3":
            category = raw_input("분류를 입력하세요\n")
            
            body['query']['match']['category'] = category
            
            results = es.search(index="naver", body=body)
            businessId = {}
            for result in results['hits']['hits']:
                businessId[result['_source']['businessId']] = True
            del body['query']['match']['category']
            
            
            query = raw_input("질의어를 입력하세요.")  
            body['query']['match']['body'] = query

            results = es.search(index="google", body=body)
            for result in results['hits']['hits']:
                if result['_source']['businessId'] in businessId:
                    print("명칭: ",result['_source']['name'])
                    print("점수: ",result['_source']['rating'])
                    print("후기글: ",result['_source']['body'].encode("utf-8").replace('\n',' '))
                    print("=============================================================================================================")
            del body['query']['match']['body']
            
        elif case is "4":
            store = raw_input("가게이름을 입력하세요.\n")
            
            body['query']['match']['name'] = store
            
            results = es.search(index="naver", body=body)
            businessId = {}
            for result in results['hits']['hits']:
                businessId[result['_source']['businessId']] = True           
            del body['query']['match']['name']
            
            
            for ID in list(businessId):
                body['query']['match']['businessId'] = ID
                results = es.search(index="google", body=body)
                for result in results['hits']['hits']:
                    if result['_source']['businessId'] in businessId:
                        print("명칭: ",result['_source']['name'])
                        print("점수: ",result['_source']['rating'])
                        print("후기글: ",result['_source']['body'].encode("utf-8").replace('\n',' '))
                        print("=============================================================================================================")
            del body['query']['match']['businessId']
            
            query = raw_input("가게의 후기를 입력하세요\n")
            body['query']['match']['body'] = query
            results = es.search(index="google", body=body)
            for result in results['hits']['hits']:
                if result['_source']['businessId'] in businessId:
                    print("명칭: ",result['_source']['name'])
                    print("점수: ",result['_source']['rating'])
                    print("후기글: ",result['_source']['body'].encode("utf-8").replace('\n',' '))
                    print("=============================================================================================================")
            del body['query']['match']['body']
        
        elif case == 'exit':
            break

In [101]:
search()

볼 목록을 고르세요. 
 1: 후기 
 2: 위치별 후기 
 3: 종목별 후기 
 4: 가게별 후기 
 5: 점수별 후기 
 나가려면 exit4
가게이름을 입력하세요.
박물관
명칭:  문화이발관
점수:  5.0
후기글:  한번 오셔서 드셔 보세요
명칭:  문화이발관
점수:  3.0
후기글:  
명칭:  무흘구곡전시관
점수:  5.0
후기글:  중국의 무이구곡과 비슷하다고 한강 정구 선생님이 …더보기
명칭:  김천세계도자기박물관
점수:  5.0
후기글:  많지는 않지만 우리전통  도자기와 세계의  그릇들이 전시되어있어요. 입장료 천원에 박물관 위아래 전체를 관람할수있어서 괜찮네요.
명칭:  김천세계도자기박물관
점수:  4.0
후기글:  자그마한 내부공간에 소량의 해외도자기류가 전시되어 있는데 각나라의 도자기 특성을 알수있는정도 입장료는 1000원이며 여기 입장권으로 바로인근에있는 시립박물관도 같이 관람가능 내부는 사진촬영 금지입니다
명칭:  김천세계도자기박물관
점수:  3.0
후기글:  깨끗하고 쾌적하고 산책하기 아주좋다 김천시장이 돈 엄청 들인듯 하다 …더보기
명칭:  김천세계도자기박물관
점수:  5.0
후기글:  이런 전통의 가치를 지키고 보호하는건 아주 훌륭함. 전통과 현대가 대립하지않고 예술로 승화해나가는 방향으로 ..
명칭:  김천세계도자기박물관
점수:  5.0
후기글:  조용하고 산책하기 좋습니다 아이들 뛰어 놀기도 좋아요
명칭:  김천세계도자기박물관
점수:  5.0
후기글:  직원이 아주 친절하셔서 기억에 남네요~^^
명칭:  김천세계도자기박물관
점수:  3.0
후기글:  밖에서만 관람..해서 경험불가.
명칭:  김천세계도자기박물관
점수:  4.0
후기글:  김천박물관  크게 좋지는 않았어요
명칭:  김천세계도자기박물관
점수:  5.0
후기글:  가족들과 꼭 가볼만한곳! 김천 직지사 가을 단풍이 정말 예쁜곳
명칭:  김천세계도자기박물관
점수:  3.0
후기글:  세계도자기 구경도 하고 유래도 잘 설명되어 있어요 관람료 천원 입니다 ㅎ
명칭:  김천세

가게의 후기를 입력하세요
놀거리
명칭:  김천녹색미래과학관
점수:  4.0
후기글:  놀거리 볼거리는 있어요. 애기들에따라 무서워 할만한 요소들이 있내요 갑자기 큰소리가 난다던지
명칭:  김천녹색미래과학관
점수:  4.0
후기글:  초등저학년 놀기 좋음
명칭:  구미 탄소제로교육관
점수:  5.0
후기글:  아이들 놀기 좋은 곳
명칭:  성주추억박물관
점수:  5.0
후기글:  아기자기하게 체험할거리가 풍성~
명칭:  김천녹색미래과학관
점수:  4.0
후기글:  취학전 애들이 놀기에 좋네요
명칭:  김천녹색미래과학관
점수:  5.0
후기글:  시설도 깨끗하고 놀거리도 많아요!!!
명칭:  김천녹색미래과학관
점수:  4.0
후기글:  애들 놀수있는 장소가 괜찮음
명칭:  김천녹색미래과학관
점수:  5.0
후기글:  저렴하고 실내에서 애들하고 놀기 너무 좋아요
명칭:  김천녹색미래과학관
점수:  5.0
후기글:  재밌는 태야광 로봇 제작 식물등의 볼거리와 체험이 있다
명칭:  김천녹색미래과학관
점수:  5.0
후기글:  아이들과 다녀오면 좋음. 공부도 되고 즐길거리가 충분함
명칭:  김천세계도자기박물관
점수:  5.0
후기글:  조용하고 산책하기 좋습니다 아이들 뛰어 놀기도 좋아요
명칭:  김천녹색미래과학관
점수:  4.0
후기글:  볼거많고 놀이방, 블럭방도 있어서 아기들 놀기도 좋아요
명칭:  김천녹색미래과학관
점수:  4.0
후기글:  집이랑 가까워서 애들데리고 자주가요 애들이랑 반나절 놀기 딱좋아요
명칭:  김천녹색미래과학관
점수:  4.0
후기글:  다양한 볼거리와 체험들이 많아서 즐거운 시간 보냈습니다 어른이 가도 재밌었어요
명칭:  김천녹색미래과학관
점수:  5.0
후기글:  아이들이 와서 체험하고 놀기 좋아요. 10세미만 아동은 실내놀이터 갈 필요 없이 이곳에서 체험도하고 놀이터에서 놀기 좋아요. 10세 이상은 게임을 할수 있는 공간이 따로 있어 좋습니다
명칭:  김천세계도자기박물관
점수:  5.0
후기글:  김천 직지사 유래 깊은 사찰로 한번 돌아

KeyboardInterrupt: 

#### search()

In [ ]:
words = ["관리","아이","체험","주차장","공원","직지사","김천","사람"]
for word in words:
    search()

In [48]:
es.indices.get_alias("*")

/home/com/anaconda3/envs/py27/lib/python2.7/site-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: this request accesses system indices: [.kibana_1, .apm-custom-link, .apm-agent-configuration, .kibana_task_manager_1], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{u'.apm-agent-configuration': {u'aliases': {}},
 u'.apm-custom-link': {u'aliases': {}},
 u'.ds-ilm-history-5-2021.03.17-000001': {u'aliases': {}},
 u'.kibana-event-log-7.11.2-000001': {u'aliases': {u'.kibana-event-log-7.11.2': {u'is_write_index': True}}},
 u'.kibana_1': {u'aliases': {u'.kibana': {}}},
 u'.kibana_task_manager_1': {u'aliases': {u'.kibana_task_manager': {}}},
 u'google': {u'aliases': {}},
 u'naver': {u'aliases': {}},
 u'store': {u'aliases': {}}}

In [41]:
es.indices.delete(index='store')

{u'acknowledged': True}

In [22]:
# 페이스북이나 인스타그램에 가입해서 #화성행궁 이라 할경우 다른곳에서 
# 우리 계정의 인증키로 
# 김천 관광이라는 페이스북 계정을 만들었을 떄
# 다른 a가 자기 계정에 해쉬태그를 공유했을 때 그 게시물을 가져올 수 있는지
# 또한, 그 계정의 다른 게시물을 가져올 수 있는지